In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from transformers import BertModel

In [2]:
# clean data
df=pd.read_csv('/kaggle/input/finaldataset/finaldataframe.csv')
df.head()

,product_name,category,product_brand
0,suavizante,Household,perla
1,suavizante doypack 3 fresca primavera,Household,suavitel
2,leche tetrabrik chocolate o frutilla,Dairy,toni
3,lavadora semiautomatica doble tanque 15 kg,Household,innova
4,"si el mundo es mejor, sabe mejor",galletas,colombina


In [3]:
df.shape

(35296, 3)

In [4]:
# Create id2label mapping
id2label = {i: label for i, label in enumerate(df['category'].unique())}
print(id2label)

{0: 'Household', 1: 'Dairy', 2: 'galletas', 3: 'Beverages', 4: 'Meat/Poultry/Seafood', 5: 'bombones', 6: 'Other', 7: 'AlcoholicBeverages', 8: 'Snacks/Candy', 9: 'Canned/JarredGoods', 10: 'PersonalCare', 11: 'FrozenFoods', 12: 'Pasta/Grains', 13: 'Bakery', 14: 'Prepared/Ready-Made Foods', 15: 'Toys: Other', 16: 'Prepared/Ready-Made_Foods', 17: 'Electronics', 18: 'Baby', 19: 'Pet', 20: 'FreshProduce', 21: 'Toys', 22: 'Produce', 23: 'Vitamins: Other', 24: 'Medicines: Household'}


In [5]:
import pandas as pd

def load_data(data_file, id2label):
    texts = df[['product_name', 'product_brand']].apply(lambda x: ' product_brand: '.join(x.dropna()), axis=1).tolist()
    labels = [key for label in df['category'] for key, value in id2label.items() if value == label]
    return texts, labels

texts, labels = load_data(df, id2label)
print(texts)
print(labels)


['suavizante product_brand: perla', 'suavizante doypack 3 fresca primavera product_brand: suavitel', 'leche tetrabrik chocolate o frutilla product_brand: toni', 'lavadora semiautomatica doble tanque 15 kg product_brand: innova', 'si el mundo es mejor, sabe mejor product_brand: colombina', 'café gold doypack product_brand: nescafé', 'jamonada product_brand: juris', "bombones fei dun estuche product_brand: ta'riko", 'jamón sanduchero product_brand: e m butidos la vienesa', 'queso holandes 80g product_brand: el caserio', 'cola naranja product_brand: mas, cola gallito', 'salchichas perro caliente product_brand: la europea', 'bombones product_brand: noggy', 'atún lomitos en aceite girasol product_brand: don sancho', 'cerveza sixpack product_brand: coronita extra', "chocolate en polvo product_brand: ta'riko", 'galletas saladas product_brand: salticas', 'aceite product_brand: sabrosón', 'leche entera o semidescremada product_brand: vita', 'margarina 450g + margarina 225g product_brand: nature

In [6]:
length=len(id2label)
print(length)

25


In [8]:
def get_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")
device=get_device()

class CategoryClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [11]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits


In [10]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

In [10]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

In [11]:
# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = length
max_length = 128
batch_size = 16
num_epochs = 4
learning_rate = 2e-5
test_size=0.2
random_state=42

In [12]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=test_size, random_state=random_state)

In [13]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = CategoryClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = CategoryClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 

The secret `HF_TOKEN` does not exist in your Colab secrets.

To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.

You will be able to reuse this secret in all of your notebooks.

Please note that authentication is recommended but still optional to access public models or datasets.

  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.

  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [15]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning

  warnings.warn(


In [16]:
train_dataloader

In [17]:
for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_dataloader, optimizer, scheduler, device)
        accuracy, report = evaluate(model, val_dataloader, device)
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(report)

Epoch 1/4


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

  _warn_prf(average, modifier, msg_start, len(result))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

  _warn_prf(average, modifier, msg_start, len(result))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 0.8299

              precision    recall  f1-score   support



           0       0.91      0.92      0.91       772

           1       0.95      0.82      0.88       649

           2       0.93      0.81      0.87        69

           3       0.87      0.91      0.89       843

           4       0.87      0.90      0.88       620

           5       1.00      0.83      0.91        24

           6       0.74      0.60      0.67       177

           7       0.89      0.93      0.91       945

           8       0.76      0.76      0.76       717

           9       0.82      0.85      0.84       397

          10       0.77      0.90      0.83       519

          11       0.45      0.13      0.21       105

          12       0.86      0.80      0.83       225

          13       0.56      0.82      0.67       333

          14       0.00      0.00      0.00        18

          15       0.00      0.00      0.00        14

          16       0.76      0.60 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

  _warn_prf(average, modifier, msg_start, len(result))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

  _warn_prf(average, modifier, msg_start, len(result))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 0.8707

              precision    recall  f1-score   support



           0       0.91      0.95      0.93       772

           1       0.94      0.91      0.92       649

           2       0.94      0.84      0.89        69

           3       0.92      0.91      0.92       843

           4       0.91      0.90      0.91       620

           5       0.95      0.83      0.89        24

           6       0.80      0.68      0.74       177

           7       0.94      0.94      0.94       945

           8       0.76      0.84      0.80       717

           9       0.86      0.88      0.87       397

          10       0.87      0.87      0.87       519

          11       0.53      0.42      0.47       105

          12       0.72      0.91      0.80       225

          13       0.76      0.81      0.79       333

          14       0.67      0.11      0.19        18

          15       0.00      0.00      0.00        14

          16       0.81      0.61 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

  _warn_prf(average, modifier, msg_start, len(result))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

  _warn_prf(average, modifier, msg_start, len(result))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 0.8796

              precision    recall  f1-score   support



           0       0.94      0.94      0.94       772

           1       0.92      0.93      0.93       649

           2       0.98      0.86      0.91        69

           3       0.89      0.93      0.91       843

           4       0.90      0.93      0.91       620

           5       0.95      0.83      0.89        24

           6       0.81      0.69      0.75       177

           7       0.93      0.94      0.94       945

           8       0.78      0.85      0.81       717

           9       0.89      0.87      0.88       397

          10       0.89      0.89      0.89       519

          11       0.68      0.44      0.53       105

          12       0.83      0.89      0.86       225

          13       0.80      0.80      0.80       333

          14       0.33      0.17      0.22        18

          15       0.00      0.00      0.00        14

          16       0.75      0.64 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

  _warn_prf(average, modifier, msg_start, len(result))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

  _warn_prf(average, modifier, msg_start, len(result))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Shopfully/FewSelectedCategory/bert_classifier.pth")

In [19]:
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [20]:
# Define the device (use GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

### Evaluating

In [12]:
# Instantiate your model
model = BERTClassifier("bert-base-uncased", length)

# Load the saved state dictionary into the model
model.load_state_dict(torch.load('/kaggle/input/modell/pytorch/model1.0/1/bert_classifier (1).pth'))
model.eval()

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [13]:
def predict_category(text, model, tokenizer, device, id2label, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)

        # Return the label corresponding to the predicted numerical ID from id2label
        return id2label[preds.item()]

In [ ]:
text = "For Children Baby 3 Months+ Concentrated Liquid Strawberry Flavour 200mg/5ml 50ml,product_brand: Nurofen"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

predicted_label = predict_category(text, model, tokenizer, device, id2label)
print("Predicted label:", predicted_label)